<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20simulation%209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt

from scipy import optimize

In [2]:
pip install MLNN

In [3]:
from MLNN import*

In [4]:
import LLC_4_1 as LLC
# import https://github.com/vladgap/LLC/LLC_4_1.py?raw=true as LLC

In [5]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_lit1
lit data with NN model (7 hidden sigmoids)

### Equilibrium literature data

In [6]:
data1=pd.DataFrame(data=[
[63.5,	1.3,	42.4,	15.0],
[58.7,	1.2,	36.9,	13.8],
[52.0, 1.0,	27.7,	11.8],
[45.5,	1.0,	18.9,	9.6],
[39.1,	1.4,	10.2,	8.3],
[31.5,	1.5,	5.1,	8.8],
[22.9,	2.0,	2.3,	9.0],
[12.6,	2.2,	0.90,	9.3],
[6.3,	2.4,	0.45,	9.45],
[0.01,	2.6,	0.01,	9.6],
],
columns=['H3PO4_aq', 'solvent_aq', 'H3PO4_org', 'water_org'])

In [7]:
data1.head()

,H3PO4_aq,solvent_aq,H3PO4_org,water_org
0,63.5,1.3,42.4,15.0
1,58.7,1.2,36.9,13.8
2,52.0,1.0,27.7,11.8
3,45.5,1.0,18.9,9.6
4,39.1,1.4,10.2,8.3


In [8]:
fig1=go.Figure()
[fig1.add_trace(go.Scatter(x=data1.H3PO4_aq, y=data1[data1.columns[i]], name=data1.columns[i])) for i in range(1,4)]
fig1.update_layout(title='', width=1200, height=600,)
fig1.show()

In [9]:
X1=np.array(data1)[:,[0]]
T1=np.array(data1)[:,[1,2,3]]

### Model
"Model for Rotem LLC simulation 1"

In [10]:
scaler_X1 = preprocessing.StandardScaler()
scaler_T1 = preprocessing.StandardScaler()
scaler_X1.mean_, scaler_X1.scale_ = [33.211], [21.13387066]
scaler_T1.mean_, scaler_T1.scale_ = [1.66,  14.486, 10.465], [0.56071383, 15.23739033,  2.16703138]

bp1=VectorBackProp(layers=[1,7,3], hidden_activation = 'sigmoid')
manual_best_weights_for_now2=[[[-4.215621245080982,
   -8.235086763118087,
   -1.6251342150606451,
   -6.9206357615871035,
   -3.4092427169369826,
   -7.4263402914028775,
   -3.7003458539301977]],
 [[0.8764395040443358,
   3.9145850159876945,
   -1.5013585830931442,
   -1.194371163565281,
   4.118411550251139,
   5.892242255497829,
   -6.755593628941706]],
 [[-2.6609909047049918, -0.44480776958732143, 1.9256391063013463],
  [2.566022222516618, -0.5042582093932455, -1.432212975592254],
  [1.405004033981333, -0.2988952466228391, 0.5807591718257896],
  [2.0692859230668117, -0.0780871568942863, -0.5145373287488922],
  [-0.7989625307881565, -1.957459884661486, -2.8991426864622065],
  [-0.5821795598385298, -0.22284877970127598, -0.9077232625087237],
  [1.619488070885094, 0.03697105802710269, 0.008613201857731975]],
 [[-0.4010006364699687, 2.4687577407660806, 2.995012357373033]]]
bp1.import_weights(manual_best_weights_for_now2)

In [11]:
class EQUIL_rotem_lit1: # literature data for pentanol 25C
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        # the model predicts %solvent_aq, %H3PO4_aq, %water_org from %H3PO4_aq
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        pred=scaler_T1.inverse_transform(bp1.run(scaler_X1.transform(x[:,[1]])))
        # prediction is column vector of lists [%solvent_aq, %H3PO4_aq, %water_org]

        self.xout = np.array([pred[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([pred[:,2], pred[:,1]]).T # [[%water_org, %H3PO4_org], ...]

## EQUIL_rotem_washing1
Rotem solvent, estimation by polynoms

In [12]:
class EQUIL_rotem_washing1: # prepared using purified and washed solvents
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        y=np.zeros_like(x)
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # y = A + B*x + C*x^2 + D*x^3

        # %solvent_aq
        D,C,B,A = -3.78E-06,8.15E-04,- 5.73E-02,2.45E+00
        x[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %water_org
        D,C,B,A = 4.45E-05,- 1.56E-03,- 1.84E-02,7.58E+00
        y[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %H3PO4_org
        # D,C,B,A = 2.00E-04,- 4.86E-03,7.07E-02,2.73E-02
        # y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3
        G,F,E,D,C,B,A = -1.97E-09,2.02E-07,- 2.95E-06,- 3.83E-05,2.60E-03,2.38E-02,1.70E-02
        y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3 + E*x[:,1]**4 + F*x[:,1]**5 + G*x[:,1]**6

        self.xout = np.array([x[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([y[:,0], y[:,1]]).T # [[%water_org, %H3PO4_org], ...]

In [13]:
# EQUIL_rotem_washing1([[2,20],[1.5,30],[1.3,40]]).xout

## EQUIL_rotem_washing2
Rotem solvent, full profile, sulfate

In [14]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org
4.15	0.147	0.0733	9.6	2.5	0.0626
14.9	0.0806	0.359	9.5	2.2	0.0852
28.6	0.0329	1.68	9.3	1.9	0.0787
45.7	0.0106	8.31	8.5	1.6	0.0664
48.2	0.0078	10.7	8.5	1.3	0.0631
49.4	0.0058	12.2	8.7	1.0	0.0641
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data2=pd.DataFrame(data[1:], columns=data[0])
data2

,H3PO4_aq,SO4_aq,H3PO4_org,water_org,solvent_aq,SO4_org
0,4.15,0.1470,0.0733,9.6,2.5,0.0626
1,14.90,0.0806,0.3590,9.5,2.2,0.0852
2,28.60,0.0329,1.6800,9.3,1.9,0.0787
3,45.70,0.0106,8.3100,8.5,1.6,0.0664
4,48.20,0.0078,10.7000,8.5,1.3,0.0631
5,49.40,0.0058,12.2000,8.7,1.0,0.0641


In [15]:
X2=np.array(data2)[:,[0,1]]
T2_1=np.array(data2)[:,[2]]
T2_2=np.array(data2)[:,[3]]
T2_3=np.array(data2)[:,[4]]
T2_4=np.array(data2)[:,[5]]

scaler_X2 = preprocessing.StandardScaler().fit(X2)
scaler_T2_1 = preprocessing.StandardScaler().fit(T2_1)
scaler_T2_2 = preprocessing.StandardScaler().fit(T2_2)
scaler_T2_3 = preprocessing.StandardScaler().fit(T2_3)
scaler_T2_4 = preprocessing.StandardScaler().fit(T2_4)

X_sc2 = scaler_X2.transform(X2)
# T_sc2_1 = scaler_T2_1.transform(T2_1)
# T_sc2_2 = scaler_T2_1.transform(T2_2)
# T_sc2_3 = scaler_T2_1.transform(T2_3)
# T_sc2_4 = scaler_T2_1.transform(T2_4)

In [16]:
class EQUIL_rotem_washing2: # from full profile, sulfate
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp2_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_1=[[[0.1735491429461148, 1.8140938189194906],
  [-0.6530409129171293, -0.6441814583921941]],
 [[0.7411897128053353, -1.2033881701371891]],
 [[0.3516552639162472], [1.6547118904069311]],
 [[-1.0860308848681413]]]
        bp2_1.import_weights(manual_weights2_1)
        predics2_1=abs(scaler_T2_1.inverse_transform(bp2_1.run(scaler_X2.transform(x[:,[1,2]]))))

        # water_org
        bp2_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_2=[[[-1.356083366626272, 2.1006456906712234],
  [-0.6309805208401018, -0.5374527149916659]],
 [[1.0515374304634164, -2.3182953246087115]],
 [[1.4841383017903245], [3.153523627853289]],
 [[-1.7074908699001965]]]
        bp2_2.import_weights(manual_weights2_2)
        predics2_2=abs(scaler_T2_2.inverse_transform(bp2_2.run(scaler_X2.transform(x[:,[1,2]]))))

        # solvent_aq
        bp2_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_3=[[[2.0994823528593702, -1.2293957752203057],
  [-0.6972915446623457, 0.5216771334283966]],
 [[-2.3459563115354527, 1.775004898861346]],
 [[-3.1726827033207408], [0.4025651395234753]],
 [[-0.4638947231253712]]]
        bp2_3.import_weights(manual_weights2_3)
        predics2_3=abs(scaler_T2_3.inverse_transform(bp2_3.run(scaler_X2.transform(x[:,[1,2]]))))

        # SO4_org
        bp2_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_4=[[[-2.7308241499437758, 1.8373933992350313],
  [-2.021898044039464, -0.8845457922101067]],
 [[1.0694417308325381, 0.08920200363835196]],
 [[2.7835653102552724], [1.6175935763910405]],
 [[-4.969235753353272]]]
        bp2_4.import_weights(manual_weights2_4)
        predics2_4=abs(scaler_T2_4.inverse_transform(bp2_4.run(scaler_X2.transform(x[:,[1,2]]))))

        self.xout = np.array([predics2_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2_2[:,0], predics2_1[:,0], predics2_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        print (self.yout, self.xout)

## EQUIL_rotem_washing3
ניסויי הפרדה עם ממס מטוהר

In [17]:
# X3=np.array(data3)[:,[0,1]]
# T3_1=np.array(data3)[:,[2]]
# T3_2=np.array(data3)[:,[3]]
# T3_3=np.array(data3)[:,[4]]
# T3_4=np.array(data3)[:,[5]]

# scaler_X3 = preprocessing.StandardScaler().fit(X3)
# scaler_T3_1 = preprocessing.StandardScaler().fit(T3_1)
# scaler_T3_2 = preprocessing.StandardScaler().fit(T3_2)
# scaler_T3_3 = preprocessing.StandardScaler().fit(T3_3)
# scaler_T3_4 = preprocessing.StandardScaler().fit(T3_4)

# X_sc3 = scaler_X3.transform(X3)

In [18]:
Xscale=[21.97576471,  0.05575882], [15.2853469,   0.05047379]
Tscale=[1.89941176, 11.19176471,  1.4,         0.02921412], [2.40216868, 1.27085536, 0.17875386, 0.01997862]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_X3_2 = preprocessing.StandardScaler().fit([[0,0]]) # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [19]:
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]])))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]]))) # מים בקלה חושבו רק על נתונים "טובים
        # predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]])))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]])))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [20]:
class EQUIL_rotem_washing3_2: # ניסויי הפרדה עם ממס מטוהר
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Literature data

### 1 - Washing



In [21]:
bat1_2=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[5,12], xin=[5,0], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])
# bat1_2=LLC.Battery(4, 87.6, 112.4, [0, 0], [0, 32.56], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=.7, entrainment_perc_out=10)

In [22]:
bat1_2.runs

35

In [23]:
table1_2=LLC.BatteryTableFull(bat1_2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,89.055359,85.475686,83.948464,83.262505
"y1 in, %",5.000000,8.829622,9.079322,9.354564,9.494674
"y2 in, %",12.000000,6.974399,3.352100,1.529652,0.668417
"Aq in, ton/hr",34.382667,30.833850,29.321083,28.640994,29.000000
"x1 in, %",2.075036,2.268582,2.424578,2.529920,5.000000
"x2 in, %",19.948914,10.447329,4.783224,1.865826,0.000000
"Org out, ton/hr",89.055213,85.474791,83.948421,83.262549,83.906581
"y1 out, %",8.829647,9.079543,9.354583,9.494752,9.558549
"y2 out, %",6.974221,3.350772,1.529553,0.667935,0.281837
"Aq out, ton/hr",34.452985,34.379990,30.833663,29.320314,28.640963


In [24]:
bat=bat1_2
fig1_3 = go.Figure()
n=1 #H3PO4
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig1_3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig1_3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig1_3.update_layout(title='', width=1200, height=600,)
fig1_3.show()

In [25]:
fig1_4=go.Figure()
current_data = data1
current_table = table1_2
[fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig1_4.update_xaxes(title_text="x, %", range=[20, 40])
# fig1_4.update_yaxes(title_text="y, %", range=[0, 15])
fig1_4.update_layout(title='', width=1200, height=600,)
fig1_4.show()

### 2 - Extraction

In [26]:
bat2=LLC.Battery(stages_num=7, Oin=90, Ain=29, yin=[5,0], xin=[0,40], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[[.8, 1], [0.5, .8], [.6,.9]])

In [27]:
bat2.runs

38

In [28]:
bat2.eff

array([[1. , 1. ],
       [1. , 0.8],
       [1. , 0.9],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ]])

In [29]:
table2=LLC.BatteryTableFull(bat2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",90.000000,96.473211,96.995114,97.591662,98.289728,99.065594,100.080908
"y1 in, %",5.000000,8.991353,8.936158,8.913425,8.882908,8.818593,8.686241
"y2 in, %",0.000000,2.343352,2.895783,3.477733,4.154708,4.927815,5.937057
"Aq in, ton/hr",24.303520,24.832050,25.420661,26.121458,26.893674,27.928541,29.000000
"x1 in, %",1.846572,1.742477,1.628782,1.519116,1.440582,1.414597,0.000000
"x2 in, %",26.076255,27.717279,29.376141,31.174464,33.199446,35.744126,40.000000
"Org out, ton/hr",96.473586,96.992375,97.593864,98.283709,99.070893,100.077211,101.186610
"y1 out, %",8.991297,8.936323,8.913326,8.883276,8.818035,8.687748,8.456033
"y2 out, %",2.343821,2.892832,3.479975,4.148719,4.933070,5.927017,7.477362
"Aq out, ton/hr",17.829934,24.312885,24.821911,25.429411,26.112510,26.916924,27.894298


In [30]:
bat=bat2
fig2_1 = go.Figure()
n=1 #H3PO4
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig2_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig2_1.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig2_1.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig2_1.update_layout(title='', width=1200, height=600,)
fig2_1.show()

In [31]:
fig2_2=go.Figure()
current_data = data1
current_table = table2
[fig2_2.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig2_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig2_2.update_yaxes(title_text="y, %", range=[0, 15])
fig2_2.update_layout(title='', width=1200, height=600,)
fig2_2.show()

## Washing data
Rotem solvent, estimation by polynoms

### 3 - Washing1

In [32]:
bat3=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1)#, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])

In [33]:
bat3.runs

141

In [34]:
table3=LLC.BatteryTable(bat3).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",204.000000,195.342174,183.280373,174.327927,172.269404,172.061591,172.039690
"y1 in, %",7.700000,7.577914,7.051886,6.835793,7.295850,7.533862,7.574132
"y2 in, %",14.400000,11.104978,6.308415,2.044015,0.446296,0.079486,0.024647
"Aq in, ton/hr",57.516594,45.446384,36.500004,34.438744,34.231179,34.209255,33.300000
"x1 in, %",1.245967,1.459202,1.968008,2.330954,2.432270,2.448012,0.000000
"x2 in, %",37.669252,25.367315,9.686259,2.142208,0.310800,0.034716,0.000000
"Org out, ton/hr",195.344416,183.275916,174.329792,172.269154,172.061607,172.039701,171.130433
"y1 out, %",7.578015,7.051702,6.835725,7.296082,7.533819,7.574140,7.579359
"y2 out, %",11.105811,6.306478,2.045084,0.445905,0.079546,0.024637,0.017830
"Aq out, ton/hr",66.172178,57.512642,45.450586,36.497517,34.438977,34.231145,34.209257


In [35]:
bat=bat3
fig3 = go.Figure()
n=1 #H3PO4
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig3.update_layout(title='', width=1200, height=600,)
fig3.show()

In [36]:
fig3_2=go.Figure()
# current_data = data1
current_table = table3
# [fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig3_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig3_2.update_yaxes(title_text="y, %", range=[0, 15])
fig3_2.update_layout(title='', width=1200, height=600,)
fig3_2.show()

### 4 - Washing2

In [37]:
bat4=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1, convergence=5e-4, eff=[1, 0.9], entrainment_perc_in=[2,0], entrainment_comp_in=[[5,5],[0,0]], entrainment_perc_out=[3,4])

In [38]:
bat4.runs

13

In [39]:
table4=LLC.BatteryTableFull(bat4).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6,Stage 7
"Org in, ton/hr",204.000000,197.476924,192.054619,184.980696,177.693848,173.096470,171.797120
"y1 in, %",7.700000,7.639709,7.413024,7.107238,6.848766,6.914708,7.272929
"y2 in, %",14.400000,11.892679,9.921331,7.179893,4.041803,1.668367,0.608861
"Aq in, ton/hr",59.968484,54.876756,47.942418,40.551758,36.028255,34.633532,33.300000
"x1 in, %",1.195117,1.234671,1.330063,1.567682,1.941366,2.272553,0.000000
"x2 in, %",42.683779,38.641698,31.823762,21.122384,10.318621,3.244250,0.000000
"Org out, ton/hr",197.489684,192.098515,185.012627,177.569184,173.104408,171.776807,169.156745
"y1 out, %",7.640308,7.414765,7.108539,6.846031,6.913954,7.285448,7.505317
"y2 out, %",11.897083,9.937810,7.192845,3.994027,1.672885,0.584278,0.169015
"Aq out, ton/hr",64.454662,60.203450,54.916699,47.892005,40.573566,35.940507,34.634218


In [40]:
bat=bat4
fig4 = go.Figure()
n=1 #H3PO4
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig4.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig4.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig4.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig4.update_layout(title='Operation Data', width=1200, height=600,)
fig4.show()

In [41]:
fig4_2=go.Figure()
# current_data = data1
current_table = table4
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig4_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig4_2.update_yaxes(title_text="y, %", range=[0, 15])
fig4_2.update_layout(title='Equilibrium Data', width=1200, height=600,)
fig4_2.show()

## Rotem washing with sulfate

### 5 - Washing1

In [42]:
EQUIL_rotem_washing2([[2.2,14.9,.0806]]).yout

[[9.56542351 0.35871824 0.08520132]] [[ 2.2011776 14.9        0.0806   ]]


array([[9.56542351, 0.35871824, 0.08520132]])

In [43]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [2, 0.1, .02], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

[[9.43019245 0.20932071 0.07283325]] [[ 2.19818904 16.21426609  0.08833174]]
[[9.30172822 1.87808801 0.08863289]] [[1.87760943e+00 2.88068189e+01 2.42557218e-02]]
[[9.67842195 2.57175191 0.13196243]] [[ 1.83723689e+00  2.68844615e+01 -8.22752546e-03]]
[[9.40774599 1.41584489 0.08787853]] [[ 1.98162    24.14670566  0.0410468 ]]
[[8.74299355 0.09728085 0.0266234 ]] [[ 2.09270285 26.11681351  0.1073246 ]]
[[ 8.71494393 10.32003728  0.05946183]] [[ 1.11956982 54.77546413  0.05751555]]
[[8.70497911 0.27831522 0.02663159]] [[ 2.02946256 28.71408958  0.0954719 ]]
[[8.73139876 0.33424516 0.02663486]] [[ 2.02535362 28.62486958  0.09290654]]
[[8.61874053 0.09909448 0.02661727]] [[ 2.06229808 28.3118691   0.1082116 ]]
[[8.628167   0.09852728 0.02661639]] [[ 2.07180798 27.88816926  0.10976343]]
[[8.60548055 0.09880625 0.0266151 ]] [[ 2.06713123 28.25772227  0.11012198]]
[[8.60813102 0.09878081 0.02661528]] [[ 2.06755767 28.21882933  0.11005313]]
[[8.60822308 0.09877921 0.02661528]] [[ 2.06758444 2

In [44]:
test1.runs

10

In [45]:
test1.output()

Oout= 74.5042053393592
yout= [8.60822308 0.09877921 0.02661528]
yout_tag= [8.60822308 0.09877921 0.02661528]  %water_org_equilib, %solubles_equilib
Aout= 44.49579466064079
xout= [ 2.06758444 28.21705147  0.11005343]
equilibrium efficiency= [1. 1. 1.]
error= [4.11296949e-05 5.50745531e-05 6.80630206e-05]
convergence= [-0.00040428 -0.00022316 -0.00016548]


In [46]:
test1 = LLC.Stage(90, 29, [8.60822308, 0.09877921, 0.02661528], [2.06758444, 28.21705147, 0.11005343], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

[[9.40587212 0.08828243 0.06697643]] [[ 2.24355622 14.81172832  0.10004056]]
[[9.64682125 2.83556223 0.13886871]] [[ 1.78411875e+00  2.90630819e+01 -1.82035537e-02]]
[[8.21751925 0.08034712 0.02647194]] [[ 2.37666577 25.232862    0.23555299]]
[[8.21268012 0.05489456 0.02621698]] [[ 2.80349145 26.46953754  0.44614586]]
[[8.74664907 0.11174503 0.02662841]] [[ 2.06769155 26.97713024  0.10147547]]
[[8.60768029 0.09853519 0.02661446]] [[ 2.07167602 28.0758201   0.11100685]]
[[8.60564232 0.09871973 0.02661485]] [[ 2.06858194 28.20448409  0.11043802]]
[[8.60879365 0.09878278 0.02661534]] [[ 2.06752461 28.2140926   0.11000452]]
[[8.60794753 0.09877988 0.02661526]] [[ 2.06757328 28.21991049  0.11006804]]


In [47]:
bat5_1=LLC.Battery(stages_num=4, Oin=90, Ain=29, yin=[8.60822308, 1, 0.05], xin=[2.06758444, 30, 0.3], EQUIL=EQUIL_rotem_washing2)

[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]
[[8.21542165 0.07060956 0.02631295]] [[ 2.53998164 31.47867061  0.35630376]]
[[8.21539376 0.07041787 0.02631317]] [[ 2.54319538 31.28743445  0.35649378]]
[[8.21539538 0.07041956 0.02631351]] [[ 2.543167   31.25688796  0.35626778]]
[[8.21539487 0.07041688 0.02631348]] [[ 2.54321195 31.25704348  0.35629013]]
[[8.21539496 0.07041739 0.02631349]] [[ 2.54320338 31.25703412  0.35628601]]
[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]
[[8.21542165 0.07060956 0.02631295]] [[ 2.53998164 31.47867061  0.35630376]]
[[8.21539376 0.07041787 0.02631317]] [[ 2.54319538 31.28743445  0.35649378]]
[[8.21539538 0.07041956 0.02631351]] [[ 2.543167   31.25688796  0.35626778]]
[[8.21539487 0.07041688 0.02631348]] [[ 2.54321195 31.25704348  0.35629013]]
[[8.21539496 0.07041739 0.02631349]] [[ 2.54320338 31.25703412  0.35628601]]
[[8.21653274 0.07325848 0.02649106]] [[ 2.49550578 17.11028579  0.23526643]]

In [48]:
bat5_1.runs

6

In [49]:
table5=LLC.BatteryTable(bat5_1).data

,Stage 1,Stage 2,Stage 3,Stage 4
"Org in, ton/hr",90.000000,88.607990,88.615281,88.615270
"y1 in, %",8.608223,8.215397,8.216589,8.216587
"y2 in, %",1.000000,0.070425,0.076451,0.076442
"y3 in, %",0.050000,0.026314,0.026386,0.026385
"Aq in, ton/hr",29.114540,29.121741,29.121729,29.000000
"x1 in, %",2.442020,2.442172,2.442171,2.067584
"x2 in, %",29.864179,29.874934,29.874918,30.000000
"x3 in, %",0.298710,0.298857,0.298856,0.300000
"Org out, ton/hr",88.607990,88.615281,88.615270,88.493540
"y1 out, %",8.215397,8.216589,8.216587,8.216587


In [50]:
bat=bat5_1
fig5 = go.Figure()
n=1 #H3PO4
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig5.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig5.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig5.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig5.update_layout(title='Operation Data', width=1200, height=600,)
fig5.show()

In [51]:
fig_equil2 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.SO4_org, mode='markers', name='SO4_org'), 2, 2)

bat=bat5_1
# H3PO4_org
fig_equil2.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', name='Efficiency'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])), y=np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])), z=np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])),
                                mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),
                                                                                                                            np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])),
                                                                                                                            np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])))],
                                textposition ='bottom center', name='Operation', marker_color='green'),1,1)

fig_equil2.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil2.update_layout(
    scene1_zaxis_range=[0,15],
    scene2_zaxis_range=[8,10],
    scene3_zaxis_range=[0,3],
    scene4_zaxis_range=[.05,.1],
    )

fig_equil2.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))

fig_equil2.show()

## Rotem washing LLC tests

### data

In [52]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
data3_names=CopyPasteToArray('''
wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
''')

In [53]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [54]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [55]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics3_1=mesh_predict_y[:,:,1]
mesh_predics3_2=mesh_predict_y[:,:,0]
mesh_predics3_3=mesh_predict_x[:,:,0]
mesh_predics3_4=mesh_predict_y[:,:,2]

### test

In [56]:
EQUIL_rotem_washing3([[2.2,10.38,.099]]).yout

array([[9.02976303, 0.5493118 , 0.01802152]])

In [57]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [0, 0.1, .02], EQUIL_rotem_washing3, eff=.8)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)
test1.runs

3

In [58]:
test1.output()

Oout= 78.32179881063443
yout= [9.60912967 3.76230026 0.03045747]
yout_tag= [9.60912967 1.20287533 0.02057183]  %water_org_equilib, %solubles_equilib
Aout= 40.67820118936556
xout= [ 1.25110892 23.80920178  0.11050305]
equilibrium efficiency= [1.  0.8 0.8]
error= [ 7.39425190e-07 -9.69302553e-04 -2.30061189e-04]
convergence= [-2.05215689e-04 -7.82960672e-04 -8.59080721e-05]


### 6 - Washing1 - water_org model

In [59]:
bat6_1=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3, eff=.7)
bat6_1.runs

21

In [60]:
table6_1=LLC.BatteryTable(bat6_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,80.315656,77.402606,76.362880,75.972655
"y1 in, %",10.000000,9.556306,9.657001,9.716974,9.747376
"y2 in, %",14.000000,5.219303,1.992958,0.760381,0.281593
"y3 in, %",0.070000,0.036196,0.023426,0.018414,0.016422
"Aq in, ton/hr",34.016975,31.103606,30.063793,29.673346,29.000000
"x1 in, %",1.626419,1.610304,1.602890,1.599657,0.000000
"x2 in, %",12.201220,4.825139,1.792163,0.579511,0.100000
"x3 in, %",0.067926,0.039113,0.026924,0.021934,0.020000
"Org out, ton/hr",80.315643,77.402535,76.362846,75.972510,75.299319
"y1 out, %",9.556313,9.657020,9.716978,9.747382,9.761492


In [61]:
fig_equil6_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat6_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil6_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil6_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil6_1.update_scenes(camera_projection_type="orthographic")

fig_equil6_1.show()

### 7 - Washing2 - water_org=12

In [62]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [63]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [64]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_2([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_2([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [65]:
bat7_1=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3_2, eff=.7)
bat7_1.runs

21

In [66]:
table7_1=LLC.BatteryTable(bat7_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",90.000000,82.794901,79.586005,78.421448,77.976314
"y1 in, %",10.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",14.000000,5.267622,2.018177,0.773599,0.288009
"y3 in, %",0.070000,0.036412,0.023539,0.018473,0.016451
"Aq in, ton/hr",34.530811,31.321228,30.156551,29.711021,29.000000
"x1 in, %",1.627135,1.610703,1.603071,1.599715,0.000000
"x2 in, %",12.499448,4.982271,1.859997,0.601322,0.100000
"x3 in, %",0.069119,0.039751,0.027203,0.022024,0.020000
"Org out, ton/hr",82.794879,79.585875,78.421405,77.976065,77.265306
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000


In [67]:
fig_equil7_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat7_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil7_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil7_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil7_1.update_scenes(camera_projection_type="orthographic")

fig_equil7_1.show()

### 8 - Washing3 - compare with plant data

Profile data

In [122]:
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])

Plant data

In [127]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

Purified solvent flow = 159.2 ton/hr,   Washed solvent flow = 134.2 ton/hr   P2O5 recovery = 98.5%


In [128]:
summary_table8=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table8

,Plant data
PS_flow,159.196491
PS_P2O5,11.6
WS_flow,134.196491
WS_P2O5,0.2
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,98.5%
Stages_num,6
Efficiency,---


In [154]:
bat8_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_2, eff=.761, convergence=1E-3, entrainment_perc_out=[3.2,0])
bat8_1.runs

53

In [155]:
bat=bat8_1
summary_table8['Model data-0.75-3']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff[0,1], str(bat.EQUIL)[17:-2]]
summary_table8

,Plant data,Model data-0.75-3
PS_flow,159.196491,159.196491
PS_P2O5,11.6,11.6
WS_flow,134.196491,130.235661
WS_P2O5,0.2,0.202975
W_flow,23.4,23.4
Prod_flow,48.4,48.207349
Prod_P2O5,37.6,37.575463
P2O5_recovery,98.5%,98.1%
Stages_num,6,6
Efficiency,---,0.761


In [156]:
table8_1=LLC.BatteryTable(bat8_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",159.196491,148.554250,141.569733,134.991122,132.539510,131.302714
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,10.954722,7.107641,3.069180,1.462465,0.689293
"y3 in, %",0.070000,0.088162,0.070161,0.038295,0.024446,0.019123
"Aq in, ton/hr",42.316079,35.113144,28.304870,25.787059,24.500154,23.400000
"x1 in, %",0.146119,0.416162,1.156770,1.658311,1.622700,0.000000
"x2 in, %",43.135071,32.853257,17.893128,8.566608,3.135096,0.000000
"x3 in, %",0.287794,0.267644,0.157185,0.079028,0.039486,0.020000
"Org out, ton/hr",148.551571,141.571936,134.981010,132.545366,131.297157,130.235661
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


In [157]:
## P2O5
bat=bat8_1
summary_table=summary_table8
fig8_1 = go.Figure()
n=1 #H3PO4
fig8_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig8_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig8_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig8_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig8_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
fig8_1.update_layout(title='', width=1200, height=600,)
fig8_1.show()

In [158]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [159]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [160]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_2([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_2([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [161]:
fig_equil8_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat8_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil8_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil8_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil8_1.update_scenes(camera_projection_type="orthographic")

fig_equil8_1.show()

### 9 - Washing4 - minimize

Plant data

In [78]:
PS_P2O5=11.6
WS_P2O5=0.5
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

Purified solvent flow = 162.8 ton/hr,   Washed solvent flow = 137.8 ton/hr   P2O5 recovery = 96.4%


In [79]:
summary_table9=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table9

,Plant data
PS_flow,162.823423
PS_P2O5,11.6
WS_flow,137.823423
WS_P2O5,0.5
W_flow,23.4
Prod_flow,48.4
Prod_P2O5,37.6
P2O5_recovery,96.4%
Stages_num,6
Efficiency,---


#### minimize1

In [116]:
PS_P2O5=11.6
WS_P2O5=0.5
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6
PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)

def P2O5_calc(x):
  eff, entrain = x[0], x[1]
  bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_2, eff=eff, convergence=1E-2, entrainment_perc_out=[entrain,0])
  error1=abs((WS_P2O5-bat.yout[1]/98*71)/WS_P2O5)
  error2=abs((Prod_P2O5-bat.xout[1]/98*71)/Prod_P2O5)
  print (eff, entrain, bat.yout[1]/98*71, bat.xout[1]/98*71, error1, error2, error1+error2, bat.runs, ',')

  return error1+error2

In [117]:
bounds=[(0.5,1),(0,5)]
sol9=optimize.direct(P2O5_calc, bounds=bounds, f_min=0, f_min_rtol=0.002, maxiter=100, locally_biased=False)

0.75 2.5 0.1952293909770379 37.413014648733714 0.6095412180459242 0.004973014661337424 0.6145142327072616 54 ,
0.9166666666666666 2.5 0.06254178727933608 37.662636598545525 0.8749164254413279 0.001665866982593716 0.8765822924239216 51 ,
0.5833333333333334 2.5 0.5836609500129385 37.000727206213014 0.167321900025877 0.015938106217739022 0.18326000624361602 36 ,
0.75 4.166666666666666 0.34719755292167703 38.712513617212245 0.30560489415664593 0.029588128117346908 0.33519302227399284 56 ,
0.75 0.8333333333333335 0.13108297982799003 36.07400261293109 0.73783404034402 0.04058503689013065 0.7784190772341506 34 ,
0.5833333333333334 4.166666666666666 0.7692172216347741 38.22634432777903 0.5384344432695483 0.016658093823910376 0.5550925370934586 42 ,
0.5833333333333334 0.8333333333333335 0.44429252572370226 35.7205507500128 0.11141494855259548 0.049985352393276686 0.16140030094587215 26 ,
0.9166666666666666 4.166666666666666 0.13315641085576027 38.94182570804468 0.7336871782884795 0.035686853937

In [106]:
print (sol9.x, sol9.fun) #[0.62654321 2.87722908]

[0.63271605 3.2133059 ] 0.001851694345967044


In [84]:
print (sol9.x, sol9.fun) #[0.62654321 2.87722908]

[0.62654321 2.87722908] 0.0017462100512864087


#### table

In [114]:
bat9_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_2, eff=0.6, convergence=1E-3, entrainment_perc_out=[2.5,0])
bat9_1.runs

48

In [115]:
bat=bat9_1
summary_table9['Model data-0.75-3']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff[0,1], str(bat.EQUIL)[17:-2]]
summary_table9

,Plant data,Model data-0.75-3
PS_flow,162.823423,162.823423
PS_P2O5,11.6,11.6
WS_flow,137.823423,133.860872
WS_P2O5,0.5,0.521264
W_flow,23.4,23.4
Prod_flow,48.4,49.025692
Prod_P2O5,37.6,36.901111
P2O5_recovery,96.4%,95.8%
Stages_num,6,6
Efficiency,---,0.6


In [88]:
table9_1=LLC.BatteryTable(bat9_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5,Stage 6
"Org in, ton/hr",162.823423,153.525326,146.797208,140.642015,136.911146,134.920201
"y1 in, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
"y2 in, %",16.011268,11.838100,8.452909,4.975206,2.665002,1.414874
"y3 in, %",0.070000,0.084940,0.074997,0.051811,0.032998,0.023953
"Aq in, ton/hr",44.254500,37.282103,30.980298,27.095245,25.056693,23.400000
"x1 in, %",0.174623,0.353807,0.879640,1.450635,1.642446,0.000000
"x2 in, %",44.473926,35.627691,23.778300,12.770545,5.160127,0.000000
"x3 in, %",0.275560,0.270628,0.197321,0.109965,0.054040,0.020000
"Org out, ton/hr",153.530407,146.779463,140.654721,136.900634,134.921863,133.316051
"y1 out, %",12.000000,12.000000,12.000000,12.000000,12.000000,12.000000


In [89]:
## P2O5
bat=bat9_1
summary_table=summary_table9
fig9_1 = go.Figure()
n=1 #H3PO4
fig9_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig9_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig9_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig9_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))

fig9_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig9_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
fig9_1.update_layout(title='', width=1200, height=600,)
fig9_1.show()

#### viz

In [107]:
data=CopyPasteToArray('''
x	y	z
0.75	2.5	0.614514233
0.916666667	2.5	0.876582292
0.583333333	2.5	0.183260006
0.75	4.166666667	0.335193022
0.75	0.833333333	0.778419077
0.583333333	4.166666667	0.555092537
0.583333333	0.833333333	0.161400301
0.916666667	4.166666667	0.769374032
0.916666667	0.833333333	0.994821279
0.638888889	0.833333333	0.441905002
0.527777778	0.833333333	0.320158563
0.583333333	1.388888889	0.101215939
0.583333333	0.277777778	0.229586672
0.638888889	1.388888889	0.365143611
0.527777778	1.388888889	0.396699532
0.638888889	2.5	0.167848488
0.527777778	2.5	0.594110028
0.583333333	3.055555556	0.317308332
0.583333333	1.944444444	0.037715995
0.601851852	1.388888889	0.220385643
0.564814815	1.388888889	0.101644713
0.583333333	1.574074074	0.094032532
0.583333333	1.203703704	0.115958079
0.638888889	1.944444444	0.296002861
0.527777778	1.944444444	0.501132981
0.638888889	0.277777778	0.496220675
0.527777778	0.277777778	0.273692306
0.601851852	1.944444444	0.076784088
0.564814815	1.944444444	0.173395039
0.583333333	2.12962963	0.116820365
0.583333333	1.759259259	0.062085329
0.638888889	3.055555556	0.04242351
0.527777778	3.055555556	0.646544011
0.601851852	1.759259259	0.18550989
0.564814815	1.759259259	0.135449036
0.589506173	1.944444444	0.077190516
0.577160494	1.944444444	0.072363042
0.583333333	2.00617284	0.03919821
0.583333333	1.882716049	0.058376419
0.805555556	4.166666667	0.516308572
0.694444444	4.166666667	0.126553822
0.75	4.722222222	0.261739652
0.75	3.611111111	0.441320452
0.657407407	3.055555556	0.134913776
0.62037037	3.055555556	0.073296142
0.638888889	3.240740741	0.011975084
0.638888889	2.87037037	0.078850803
0.589506173	2.00617284	0.041289776
0.577160494	2.00617284	0.078838576
0.585390947	1.944444444	0.053898834
0.58127572	1.944444444	0.043928175
0.583333333	1.965020576	0.042659521
0.583333333	1.923868313	0.039846089
0.694444444	4.722222222	0.044359016
0.694444444	3.611111111	0.218995443
0.657407407	3.240740741	0.09847013
0.62037037	3.240740741	0.110969691
0.805555556	4.722222222	0.438181943
0.805555556	3.611111111	0.596896581
0.712962963	4.722222222	0.112069833
0.675925926	4.722222222	0.13544978
0.694444444	4.907407407	0.096319258
0.694444444	4.537037037	0.057737795
0.645061728	3.240740741	0.034312794
0.632716049	3.240740741	0.030253487
0.638888889	3.302469136	0.026988785
0.638888889	3.179012346	0.014861173
0.638888889	4.166666667	0.224979086
0.527777778	4.166666667	0.990016777
0.583333333	4.722222222	0.67267735
0.583333333	3.611111111	0.431912359
0.712962963	4.537037037	0.136270571
0.675925926	4.537037037	0.090005335
0.645061728	3.179012346	0.045315004
0.632716049	3.179012346	0.032018316
0.640946502	3.240740741	0.012373988
0.636831276	3.240740741	0.020499081
0.638888889	3.261316872	0.017073266
0.638888889	3.220164609	0.006777933
0.638888889	4.722222222	0.27203834
0.638888889	3.611111111	0.098080915
0.657407407	2.87037037	0.176616981
0.62037037	2.87037037	0.037283016
0.645061728	3.302469136	0.022086984
0.632716049	3.302469136	0.052363406
0.640946502	3.220164609	0.016359832
0.636831276	3.220164609	0.014873415
0.805555556	2.5	0.730307953
0.694444444	2.5	0.416838084
0.75	3.055555556	0.530838967
0.75	1.944444444	0.667907976
0.601851852	1.574074074	0.203768324
0.564814815	1.574074074	0.1175789
0.62654321	2.87037037	0.00245182
0.614197531	2.87037037	0.075089046
0.62037037	2.932098765	0.049273381
0.62037037	2.808641975	0.025314899
0.640946502	3.179012346	0.024473181
0.636831276	3.179012346	0.005015125
0.638888889	3.199588477	0.010480591
0.638888889	3.158436214	0.019317285
0.63957476	3.220164609	0.009570845
0.638203018	3.220164609	0.009630134
0.638888889	3.22702332	0.008515178
0.638888889	3.213305898	0.007649529
0.694444444	3.055555556	0.322642387
0.694444444	1.944444444	0.511499482
0.712962963	4.907407407	0.090570821
0.675925926	4.907407407	0.158137801
0.62654321	2.932098765	0.010520379
0.62654321	2.808641975	0.017014766
0.805555556	3.055555556	0.677678964
0.805555556	1.944444444	0.78489526
0.657407407	3.611111111	0.030702147
0.62037037	3.611111111	0.18385846
0.638888889	3.796296296	0.140220311
0.638888889	3.425925926	0.055039039
0.645061728	3.055555556	0.070148933
0.632716049	3.055555556	0.011007829
0.638888889	3.117283951	0.028976866
0.638888889	2.99382716	0.056818166
0.628600823	2.87037037	0.014472321
0.624485597	2.87037037	0.011069502
0.62654321	2.890946502	0.002778138
0.62654321	2.849794239	0.007108151
0.972222222	4.166666667	0.843511733
0.861111111	4.166666667	0.661046272
0.916666667	4.722222222	0.683614528
0.916666667	3.611111111	0.82794572
0.657407407	3.796296296	0.043077452
0.657407407	3.425925926	0.064635481
0.632716049	3.117283951	0.013962536
0.632716049	2.99382716	0.016651215
0.628600823	2.890946502	0.011810258
0.624485597	2.890946502	0.015029959
0.627229081	2.87037037	0.006333549
0.625857339	2.87037037	0.004498244
0.62654321	2.877229081	0.00174621
0.62654321	2.86351166	0.004042849
''')
viz1=pd.DataFrame(data[1:], columns=data[0])

In [110]:
data=CopyPasteToArray('''
x	y	z
0.75	2.5	0.615793319
0.916666667	2.5	0.878114527
0.583333333	2.5	0.171870997
0.75	4.166666667	0.350964271
0.75	0.833333333	0.787947631
0.583333333	4.166666667	0.524982065
0.583333333	0.833333333	0.178445994
0.916666667	4.166666667	0.759799093
0.916666667	0.833333333	0.997180914
0.638888889	2.5	0.19620702
0.527777778	2.5	0.609068888
0.583333333	3.055555556	0.282481979
0.583333333	1.944444444	0.061126319
0.638888889	1.944444444	0.310844226
0.527777778	1.944444444	0.490852263
0.638888889	0.833333333	0.448533697
0.527777778	0.833333333	0.31316021
0.583333333	1.388888889	0.10526513
0.583333333	0.277777778	0.247477981
0.601851852	1.944444444	0.108868455
0.564814815	1.944444444	0.194417478
0.583333333	2.12962963	0.096628219
0.583333333	1.759259259	0.046503394
0.638888889	1.388888889	0.381576687
0.527777778	1.388888889	0.374373392
0.601851852	1.759259259	0.15343996
0.564814815	1.759259259	0.156174305
0.638888889	0.277777778	0.512795606
0.527777778	0.277777778	0.258408116
0.601851852	2.12962963	0.076908215
0.564814815	2.12962963	0.229262271
0.589506173	1.759259259	0.084187475
0.577160494	1.759259259	0.070582637
0.583333333	1.820987654	0.04132116
0.583333333	1.697530864	0.05722207
0.638888889	3.055555556	0.0691958
0.527777778	3.055555556	0.720664261
0.601851852	1.388888889	0.20749097
0.564814815	1.388888889	0.099628648
0.583333333	1.574074074	0.077880366
0.583333333	1.203703704	0.129833068
0.589506173	1.820987654	0.068556898
0.577160494	1.820987654	0.080277161
0.805555556	4.166666667	0.521600383
0.694444444	4.166666667	0.131173497
0.75	4.722222222	0.259483252
0.75	3.611111111	0.442014397
0.657407407	3.055555556	0.161687951
0.62037037	3.055555556	0.044025771
0.638888889	3.240740741	0.026498629
0.638888889	2.87037037	0.110585473
0.589506173	1.697530864	0.082594123
0.577160494	1.697530864	0.054307729
0.585390947	1.820987654	0.042945881
0.58127572	1.820987654	0.057474015
0.583333333	1.841563786	0.042078188
0.583333333	1.800411523	0.040127617
0.694444444	4.722222222	0.034665576
0.694444444	3.611111111	0.229568198
0.657407407	3.240740741	0.124568403
0.62037037	3.240740741	0.079455384
0.805555556	4.722222222	0.439895545
0.805555556	3.611111111	0.605810984
0.712962963	4.722222222	0.116148363
0.675925926	4.722222222	0.115388267
0.694444444	4.907407407	0.075525728
0.694444444	4.537037037	0.068153546
0.645061728	3.240740741	0.058677464
0.632716049	3.240740741	0.008508589
0.638888889	3.302469136	0.014643949
0.638888889	3.179012346	0.035396061
0.638888889	4.166666667	0.183917217
0.527777778	4.166666667	0.959518531
0.583333333	4.722222222	0.654971395
0.583333333	3.611111111	0.393103579
0.712962963	4.537037037	0.141533086
0.675925926	4.537037037	0.076967537
0.632716049	3.302469136	0.020439228
0.632716049	3.179012346	0.012846559
0.638888889	4.722222222	0.310477309
0.638888889	3.611111111	0.056238902
0.712962963	4.907407407	0.088890747
0.675925926	4.907407407	0.159778732
0.700617284	4.722222222	0.063505281
0.688271605	4.722222222	0.06017458
0.694444444	4.783950617	0.048254238
0.694444444	4.660493827	0.046521082
0.634773663	3.240740741	0.006254111
0.630658436	3.240740741	0.020486707
0.632716049	3.261316872	0.010546768
0.632716049	3.220164609	0.004006889
0.805555556	2.5	0.73288688
0.694444444	2.5	0.444493501
0.75	3.055555556	0.531139415
0.75	1.944444444	0.677071712
0.657407407	3.611111111	0.051899815
0.62037037	3.611111111	0.16028634
0.638888889	3.796296296	0.101216171
0.638888889	3.425925926	0.016212588
0.62654321	3.055555556	0.008494281
0.614197531	3.055555556	0.083402072
0.62037037	3.117283951	0.054602154
0.62037037	2.99382716	0.034863408
0.634773663	3.220164609	0.012682138
0.630658436	3.220164609	0.017258713
0.694444444	3.055555556	0.329007448
0.694444444	1.944444444	0.522449979
0.657407407	3.425925926	0.090264861
0.62037037	3.425925926	0.113403089
0.62654321	3.117283951	0.019795981
0.62654321	2.99382716	0.012838078
0.63340192	3.220164609	0.006676203
0.632030178	3.220164609	0.006541932
0.632716049	3.22702332	0.004464787
0.632716049	3.213305898	0.001851694
''')
viz2=pd.DataFrame(data[1:], columns=data[0])

In [118]:
data=CopyPasteToArray('''
x	y	z
0.75	2.5	0.614514233
0.916666667	2.5	0.876582292
0.583333333	2.5	0.183260006
0.75	4.166666667	0.335193022
0.75	0.833333333	0.778419077
0.583333333	4.166666667	0.555092537
0.583333333	0.833333333	0.161400301
0.916666667	4.166666667	0.769374032
0.916666667	0.833333333	0.994821279
0.638888889	0.833333333	0.441905002
0.527777778	0.833333333	0.320158563
0.583333333	1.388888889	0.101215939
0.583333333	0.277777778	0.229586672
0.638888889	2.5	0.167848488
0.527777778	2.5	0.594110028
0.583333333	3.055555556	0.317308332
0.583333333	1.944444444	0.037715995
0.638888889	1.388888889	0.365143611
0.527777778	1.388888889	0.396699532
0.805555556	4.166666667	0.516308572
0.694444444	4.166666667	0.126553822
0.75	4.722222222	0.261739652
0.75	3.611111111	0.441320452
0.638888889	1.944444444	0.296002861
0.527777778	1.944444444	0.501132981
0.638888889	4.166666667	0.224979086
0.527777778	4.166666667	0.990016777
0.583333333	4.722222222	0.67267735
0.583333333	3.611111111	0.431912359
0.694444444	4.722222222	0.044359016
0.694444444	3.611111111	0.218995443
0.601851852	1.944444444	0.076784088
0.564814815	1.944444444	0.173395039
0.583333333	2.12962963	0.116820365
0.583333333	1.759259259	0.062085329
0.805555556	2.5	0.730307953
0.694444444	2.5	0.416838084
0.75	3.055555556	0.530838967
0.75	1.944444444	0.667907976
0.638888889	4.722222222	0.27203834
0.638888889	3.611111111	0.098080915
0.712962963	4.722222222	0.112069833
0.675925926	4.722222222	0.13544978
0.694444444	4.907407407	0.096319258
0.694444444	4.537037037	0.057737795
0.589506173	1.944444444	0.077190516
0.577160494	1.944444444	0.072363042
0.583333333	2.00617284	0.03919821
0.583333333	1.882716049	0.058376419
0.972222222	4.166666667	0.843511733
0.861111111	4.166666667	0.661046272
0.916666667	4.722222222	0.683614528
0.916666667	3.611111111	0.82794572
0.638888889	0.277777778	0.496220675
0.527777778	0.277777778	0.273692306
0.657407407	3.611111111	0.030702147
0.62037037	3.611111111	0.18385846
0.638888889	3.796296296	0.140220311
0.638888889	3.425925926	0.055039039
0.712962963	4.537037037	0.136270571
0.675925926	4.537037037	0.090005335
0.700617284	4.722222222	0.048249333
0.688271605	4.722222222	0.077837404
0.694444444	4.783950617	0.076546212
0.694444444	4.660493827	0.037935495
0.589506173	2.00617284	0.041289776
0.577160494	2.00617284	0.078838576
0.585390947	1.944444444	0.053898834
0.58127572	1.944444444	0.043928175
0.583333333	1.965020576	0.042659521
0.583333333	1.923868313	0.039846089
0.805555556	0.833333333	0.881238414
0.694444444	0.833333333	0.652482922
0.75	1.388888889	0.730676621
0.75	0.277777778	0.826645243
0.638888889	3.055555556	0.04242351
0.527777778	3.055555556	0.646544011
0.601851852	1.388888889	0.220385643
0.564814815	1.388888889	0.101644713
0.583333333	1.574074074	0.094032532
0.583333333	1.203703704	0.115958079
0.657407407	3.796296296	0.043077452
0.657407407	3.425925926	0.064635481
0.972222222	2.5	0.921240287
0.861111111	2.5	0.818405846
0.916666667	3.055555556	0.854734859
0.916666667	1.944444444	0.919410013
0.694444444	3.055555556	0.322642387
0.694444444	1.944444444	0.511499482
0.657407407	3.055555556	0.134913776
0.62037037	3.055555556	0.073296142
0.638888889	3.240740741	0.011975084
0.638888889	2.87037037	0.078850803
0.663580247	3.611111111	0.062634633
0.651234568	3.611111111	0.034809707
0.657407407	3.672839506	0.019292183
0.657407407	3.549382716	0.041993609
0.972222222	0.833333333	1.028658338
0.861111111	0.833333333	0.939566754
0.916666667	1.388888889	0.962423191
0.916666667	0.277777778	1.021631837
0.805555556	4.722222222	0.438181943
0.805555556	3.611111111	0.596896581
0.712962963	4.166666667	0.212995721
0.675925926	4.166666667	0.041035296
0.694444444	4.351851852	0.110435754
0.694444444	3.981481481	0.150717698
0.657407407	3.240740741	0.09847013
0.62037037	3.240740741	0.110969691
0.694444444	1.388888889	0.572418157
0.694444444	0.277777778	0.697553525
0.601851852	0.833333333	0.266067122
0.564814815	0.833333333	0.053591018
0.583333333	1.018518519	0.14236337
0.583333333	0.648148148	0.181862966
0.675925926	4.351851852	0.034666983
0.675925926	3.981481481	0.062765186
0.645061728	3.240740741	0.034312794
0.632716049	3.240740741	0.030253487
0.638888889	3.302469136	0.026988785
0.638888889	3.179012346	0.014861173
0.861111111	4.722222222	0.566654314
0.861111111	3.611111111	0.726482878
0.657407407	2.5	0.271047921
0.62037037	2.5	0.067146104
0.638888889	2.685185185	0.120464411
0.638888889	2.314814815	0.210769799
0.564814815	1.018518519	0.06452056
0.564814815	0.648148148	0.073174471
0.682098765	4.351851852	0.053691787
0.669753086	4.351851852	0.062174682
0.675925926	4.413580247	0.033862764
0.675925926	4.290123457	0.02831281
0.645061728	3.179012346	0.045315004
0.632716049	3.179012346	0.032018316
0.640946502	3.240740741	0.012373988
0.636831276	3.240740741	0.020499081
0.638888889	3.261316872	0.017073266
0.638888889	3.220164609	0.006777933
0.805555556	3.055555556	0.677678964
0.805555556	1.944444444	0.78489526
0.601851852	2.5	0.063878805
0.564814815	2.5	0.319502211
0.583333333	2.685185185	0.221111665
0.583333333	2.314814815	0.163334193
0.601851852	1.759259259	0.18550989
0.564814815	1.759259259	0.135449036
0.682098765	4.166666667	0.0683913
0.669753086	4.166666667	0.038245415
0.675925926	4.228395062	0.03081673
0.675925926	4.104938272	0.040899164
0.663580247	3.672839506	0.053202127
0.651234568	3.672839506	0.048618721
0.640946502	3.179012346	0.024473181
0.636831276	3.179012346	0.005015125
0.638888889	3.199588477	0.010480591
0.638888889	3.158436214	0.019317285
0.640946502	3.220164609	0.016359832
0.636831276	3.220164609	0.014873415
0.861111111	3.055555556	0.786747662
0.861111111	1.944444444	0.861862028
0.712962963	3.611111111	0.2991168
0.675925926	3.611111111	0.129548429
0.694444444	3.796296296	0.184854229
0.694444444	3.425925926	0.252035567
0.601851852	2.685185185	0.144434968
0.601851852	2.314814815	0.027781108
0.645061728	3.055555556	0.070148933
0.632716049	3.055555556	0.011007829
0.638888889	3.117283951	0.028976866
0.638888889	2.99382716	0.056818166
0.645061728	3.302469136	0.022086984
0.632716049	3.302469136	0.052363406
0.659465021	3.672839506	0.0308953
0.655349794	3.672839506	0.026512821
0.657407407	3.693415638	0.021096798
0.657407407	3.652263374	0.022752959
0.636831276	3.199588477	0.009009746
0.636831276	3.158436214	0.011222234
0.972222222	4.722222222	0.767314846
0.972222222	3.611111111	0.888299658
0.657407407	4.166666667	0.100347516
0.62037037	4.166666667	0.32573767
0.638888889	4.351851852	0.249464451
0.638888889	3.981481481	0.182235655
0.62037037	2.685185185	0.016187885
0.62037037	2.314814815	0.110089289
0.608024691	2.314814815	0.045210906
0.595679012	2.314814815	0.076494027
0.601851852	2.37654321	0.038397806
0.601851852	2.25308642	0.021121681
0.632716049	3.117283951	0.013962536
0.632716049	2.99382716	0.016651215
0.637517147	3.179012346	0.0088678
0.636145405	3.179012346	0.005701555
0.636831276	3.185871056	0.004978035
0.636831276	3.172153635	0.006763002
0.805555556	1.388888889	0.829701297
0.805555556	0.277777778	0.925296545
0.601851852	0.277777778	0.327286833
0.564814815	0.277777778	0.119171828
0.583333333	0.462962963	0.175764841
0.583333333	0.092592593	0.248094908
0.657407407	2.87037037	0.176616981
0.62037037	2.87037037	0.037283016
0.62654321	2.685185185	0.037221341
0.614197531	2.685185185	0.038998792
0.62037037	2.74691358	0.00892845
0.62037037	2.62345679	0.03826434
0.634773663	3.055555556	0.024119973
0.630658436	3.055555556	0.007307513
0.632716049	3.076131687	0.008795809
0.632716049	3.034979424	0.012887279
0.63957476	3.220164609	0.009570845
0.638203018	3.220164609	0.009630134
0.638888889	3.22702332	0.008515178
0.638888889	3.213305898	0.007649529
0.637517147	3.185871056	0.007304694
0.636145405	3.185871056	0.007969329
0.972222222	3.055555556	0.904094004
0.972222222	1.944444444	0.956893316
0.768518519	4.722222222	0.323850351
0.731481481	4.722222222	0.19395743
0.75	4.907407407	0.232015651
0.75	4.537037037	0.289400617
0.601851852	1.574074074	0.203768324
0.564814815	1.574074074	0.1175789
0.62654321	2.87037037	0.00245182
0.614197531	2.87037037	0.075089046
0.62037037	2.932098765	0.049273381
0.62037037	2.808641975	0.025314899
0.62654321	2.74691358	0.036469974
0.614197531	2.74691358	0.044846433
0.630658436	3.076131687	0.00972439
''')
viz3=pd.DataFrame(data[1:], columns=data[0]) # locally_biased=False

In [121]:
fig = go.Figure()
fig.add_trace(go.Mesh3d(x=viz3.x, y=viz3.y, z=viz3.z, color='lightblue', opacity=0.70, name='viz1')) # locally_biased=False
fig.add_trace(go.Mesh3d(x=viz2.x, y=viz2.y, z=viz2.z, color='lightgreen', opacity=0.70))
fig.add_trace(go.Mesh3d(x=viz1.x, y=viz1.y, z=viz1.z, color='lightpink', opacity=0.70))
fig.add_trace(go.Scatter3d(x=viz3.x, y=viz3.y, z=viz3.z, mode='markers', marker_color='blue', marker_size=2, marker_opacity=0.4))
fig.show()